In [2]:
import os
import time
from os.path import join, exists
from os import listdir, makedirs
from datetime import datetime
from google import genai
from google.genai import types
from openai import OpenAI
from openai import AsyncOpenAI
import requests
import json
from pydantic import BaseModel, Field
from crawl4ai import *
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
from dataclasses import dataclass
from rich import print as rprint
from rich.console import Console
from rich.markdown import Markdown
from queue import Queue, Empty
import asyncio
import nest_asyncio 
# Add this line to allow nested event loops
nest_asyncio.apply()

from agent_tools import *
from agent_utils import *

from loguru import logger

config = Config()

console = Console()
# Log to a file with custom timestamp format
logger.add("agent_output.log", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")
model = GeminiModel(config.FLASH2_MODEL)


In [27]:
class Question(BaseModel):
    question: str = Field(description="The question that needs to be answered.")
    index: int = Field(description="The question number or index.")

class Answer(BaseModel):
   
    answer: str = Field(description="The answer for the question (if available).")
    notes: list[str] = Field(description="Add study notes or pieces of text that helps other assistants to answer the question in the future. The better the quality of the notes, the more likely it is to get better responses in the future.")
    index: int = Field(description="Equals question number or index.")
    rating: int = Field(description="Rate the quality of your response between 0 and 10", ge=0, le=10)

class Deps(BaseModel):
    questions: dict[int, Question] = Field(description="A dict with questions that need to be answered (keys: question number, values: question).")

    def get_all_questions(self) -> dict:
        return self.questions

    def remove_question(self, idx: int) -> bool:
        if idx not in self.questions.keys():
            return False

        del self.questions[idx]
        
        return True

questions_list = [
    "Explain the concept of 'artificial intelligence' in a way that a 10-year-old could understand.", 
    "Compare and contrast the philosophies of Plato and Aristotle, highlighting their key differences and similarities in their views on ethics and knowledge.", 
    "Write a short poem about the feeling of walking through a forest in autumn.",
    "If someone is planning a trip to Italy and enjoys art and history, what are three cities you would recommend they visit and why?",
    "Summarize the main arguments for and against universal basic income."]

question_dict = {}
for (k,q) in enumerate(questions_list):
    question = Question(question=q, index=k)
    question_dict[k] = question

deps = Deps(questions=question_dict)

system_prompt="""
You are a helpful assistant.

- First get all questions (use tools)
- Then, choose only one question
- Finally answer the one question
"""

agent = Agent(
    model,
    deps_type=deps,  
    result_type=Answer,
    system_prompt=system_prompt)

@agent.tool
def get_all_questions(ctx: RunContext[Deps]) -> dict:
    return ctx.deps.get_all_questions()

In [29]:
saved_answered = []

for k in range(2):
    
    result = await agent.run('Get all questions and give an answer to only one question.', deps=deps)
    # console.print(Markdown(result.data.answer))
    # logger.info(f"LLM Output: {result.data.answer}")

    saved_answered.append(result.data)

    if result.data.rating >= 8:
        # remove question from deps
        deps.remove_question(result.data.index)

    time.sleep(1)


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"


In [31]:
saved_answered

[Answer(answer="Imagine a computer that can learn and think a little bit like a human. That's kind of what artificial intelligence is! It's like teaching a computer to do things that usually only smart people can do, like understanding what you say or playing games. We use it in our phones, in video games, and in many other things!", notes=['The response provides an analogy for a 10-year-old to understand the concept of AI, focusing on learning and thinking capabilities.', 'It uses simple language and relates it to their daily life experiences like phones and video games'], index=0, rating=9),
 Answer(answer="Plato and Aristotle, while both giants of Greek philosophy, had some key differences in their approaches to ethics and knowledge. Plato, influenced by his teacher Socrates, leaned towards idealism. He believed in a world of Forms, which are perfect and eternal ideas, and that our world is just a shadow or reflection of these Forms. For Plato, true knowledge was not derived from se